In [1]:
import pandas as pd 
import numpy as np

import missingno as msno 
import miceforest as mf

from dython.nominal import associations
from dython.nominal import identify_nominal_columns
from dython.data_utils import split_hist
from dython.data_utils import identify_columns_with_na

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import style

In [2]:
#supress warnings
import warnings
warnings.filterwarnings("ignore")

# To Display all columns 
pd.set_option('display.max_columns', None)

style.use(['science', 'notebook', 'grid'])

In [3]:
dallas_city = pd.read_csv('../dataset/processed/dallas_cleaned.csv')
dallas_city.head(3)

,Unnamed: 0,Severity,Start_Time,End_Time,Duration,Start_Lat,Start_Lng,End_Lat,End_Lng,Distance(mi),Street_Name,Street_Side,Zipcode,County,Airport_Code,Weather_Timestamp,Temperature(F),Wind_Chill(F),Humidity(%),Pressure(in),Visibility(mi),Wind_Direction,Wind_Speed(mph),Precipitation(in),Weather_Condition,Amenity,Bump,Crossing,Give_Way,Junction,No_Exit,Railway,Roundabout,Station,Stop,Traffic_Calming,Traffic_Signal,Turning_Loop,Sunrise_Sunset,Civil_Twilight,Nautical_Twilight,Astronomical_Twilight
0,0,2,2016-11-30 15:29:48,2016-11-30 21:29:48,06:00:00,32.84365,-96.77463,32.81833,-96.78806,1.915,US-75 S,R,75205,Dallas,KDAL,2016-11-30 15:53:00,61.0,NaN,22.0,30.01,10.0,NNW,4.6,NaN,Clear,False,False,False,False,True,False,False,False,False,False,False,False,False,Day,Day,Day,Day
1,1,2,2016-11-30 15:41:16,2016-11-30 21:41:16,06:00:00,32.84662,-96.63736,32.84640,-96.63711,0.021,I-635 S,R,75228,Dallas,KDAL,2016-11-30 15:53:00,61.0,NaN,22.0,30.01,10.0,NNW,4.6,NaN,Clear,False,False,False,False,False,False,False,False,False,False,False,False,False,Day,Day,Day,Day
2,2,3,2016-11-30 16:16:05,2016-11-30 22:16:05,06:00:00,32.79385,-96.70278,32.79498,-96.69424,0.502,R L Thornton Fwy E,R,75228,Dallas,KDAL,2016-11-30 15:53:00,61.0,NaN,22.0,30.01,10.0,NNW,4.6,NaN,Clear,False,False,False,False,True,False,False,False,False,False,False,False,False,Day,Day,Day,Day


In [4]:
dallas_city['County'].value_counts()

Dallas     41021
Collin       383
Tarrant      130
Denton        81
Name: County, dtype: int64

In [5]:
sub = dallas_city[dallas_city['County']=='Collin']
# sub.head(3)

In [6]:
sub.rename(columns={'Number': 'Street_Number', 
                            'Street':'Street_Name', 
                            'Side': 'Street_Side'}, inplace=True)

In [7]:
sub.drop(columns=['City', 'State', 'Country', 'Timezone', 'Description' ], inplace=True)

KeyError: "['City', 'State', 'Country', 'Timezone', 'Description'] not found in axis"

In [ ]:
categorical_features=identify_nominal_columns(sub)
categorical_features

In [ ]:
# split_hist(sub, 'Street_Name', split_by='Severity', bins=20, figsize=(20,7))

In [ ]:
complete_correlation= associations(sub, filename='complete_correlation.jpeg', figsize=(30,10))

In [ ]:
df_complete_corr=complete_correlation['corr']
df_complete_corr.dropna(axis=1, how='all').dropna(axis=0, how='all').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)

In [ ]:
selected_column= sub[categorical_features]
categorical_df = selected_column.copy()

In [ ]:
categorical_correlation= associations(categorical_df, filename= 'categorical_correlation.jpeg', figsize=(20,10))

## Visualizing Missing Values

In [ ]:
identify_columns_with_na(sub)

In [ ]:
sub.shape

In [ ]:
msno.bar(sub)

In [ ]:
msno.matrix(sub)

In [ ]:
msno.heatmap(sub)

In [ ]:
msno.dendrogram(sub)

In [ ]:
sub.drop(columns=['Street_Number'], inplace=True)

In [ ]:
sub.to_csv('../dataset/processed/subset.csv')

In [ ]:
flag = ((sub['Wind_Chill(F)'].isna()==True) & (sub['Precipitation(in)'].isna()==True))

sub[flag][['Wind_Chill(F)', 'Precipitation(in)']].head(10)

In [ ]:
sub.shape

In [ ]:
df = sub[['Start_Time', 'End_Time', 'Weather_Timestamp', 'Street_Name']]
df

In [ ]:
sub.drop(columns=['Start_Time', 'End_Time', 'Weather_Timestamp', 'Street_Name'], inplace=True)

In [ ]:
sub.shape

In [ ]:
#Convert to Categorical Data 

categorical_data = sub.columns[sub.dtypes == "object"].tolist()
data = sub[categorical_data].apply(pd.Categorical)
sub[categorical_data] = data
sub.shape

In [ ]:
# imp = MiceImputer()
# imp.fit_transform(sub)

In [ ]:
kds = mf.ImputationKernel(
    sub,
    datasets=5,
    save_all_iterations=True,
    random_state=11
)

kds.mice(4)

In [ ]:
kernel = mf.ImputationKernel(
    sub,
    datasets=5,
    mean_match_candidates=5,
    save_all_iterations=True,
    random_state=11
)

kernel.mice(4)

In [ ]:
print(kds)

In [ ]:
 completed_dataset = kds.complete_data(dataset=0, inplace=False)
print(completed_dataset.isnull().sum(0))

In [ ]:
# obtain our completed dataset directly from the kernel
# kds.complete_data(dataset=0, inplace=True)
# print(kds.working_data.isnull().sum(0))

In [ ]:
completed_dataset.shape

In [ ]:
kds.plot_mean_convergence(wspace=0.75,hspace=0.75)

In [ ]:
kernel.plot_mean_convergence(wspace=0.75,hspace=0.75)

In [ ]:
# The red line is the original data, and each black line are the imputed values of each dataset.
kds.plot_imputed_distributions(wspace=1,hspace=1)

In [ ]:
# The red line is the original data, and each black line are the imputed values of each dataset.
kernel.plot_imputed_distributions(wspace=1,hspace=1)

In [ ]:
kds.plot_correlations(wspace=1,hspace=1)

In [ ]:
# kds.plot_feature_importance(dataset=0,cmap="YlGnBu",vmin=0, vmax=1 )

In [ ]:
imputed_data = pd.concat([kds.complete_data(i) for i in range(5)]).groupby(level=0).mean()
imputed_data.shape

In [ ]:
imputed = pd.merge(df, imputed_data, left_index=True, right_index=True)
imputed.shape

In [ ]:
imputed.shape

In [ ]:
complete_correlation= associations(imputed, filename='complete_correlation.jpeg', figsize=(30,10))

In [ ]:
df_complete_corr=complete_correlation['corr']
df_complete_corr.dropna(axis=1, how='all').dropna(axis=0, how='all').style.background_gradient(cmap='coolwarm', axis=None).set_precision(2)